In [10]:
from time import time
import torch
import torch.nn as nn
from torch.utils.data import SubsetRandomSampler
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [15]:
def get_train_test_loader(trainset, testset, batch_size):

    # Making dataset iterable
    train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                               batch_size=batch_size,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=testset,
                                              batch_size=batch_size,
                                              shuffle=False)

    return train_loader, test_loader

In [12]:
'''
    Parameters:
    - k, number of folds
    - trainset, the training set
    - batch size
'''
def split_data_in_k_folds(k, trainset, batch_size):
    patterns_per_fold = len(trainset) / k

    samplers = []
    for i in range(k):
        samplers.append(SubsetRandomSampler(np.arange(i * patterns_per_fold, (i + 1) * patterns_per_fold, dtype=np.int32)))

    all_data_folds = []

    for sampler in samplers:
        all_data_folds.append(
            torch.utils.data.DataLoader(dataset=trainset,
                                        batch_size=batch_size,
                                        sampler=sampler,
                                        shuffle=False)
        )

    return all_data_folds

In [42]:
# TODO: Check with Agajan how many layers we can do here...
'''
Explain the parameters here. 
in_dim -> cifar case 32x32. I am supposing those are square matrix.
d -> stands for depth.
'''
class CNNModel(nn.Module):
    def __init__(self, **params):
        super(CNNModel, self).__init__()
        # Convolution 1
        self.cnn1 = nn.Conv3d(in_channels=params['d1'], out_channels=params['d2'], kernel_size=params['f1'],
                              stride=params['s1'], padding=params['p1'])
        # Activation 1
        self.relu1 = nn.ReLU()
        # Max pooling 1
        self.maxpool1 = nn.MaxPool3d(kernel_size=2)
        # Convolution 2
        self.cnn2 =  self.cnn1 = nn.Conv3d(in_channels=params['d2'], out_channels=params['d3'], kernel_size=params['f2'],
                              stride=params['s2'], padding=params['p2'])
        # Activation 2
        self.relu2 = nn.ReLU()
        # Max pooling 2
        self.maxpool2 = nn.MaxPool3d(kernel_size=2)

        self.W2 = (params['in_dim'] - params['f1'] + 2*params['p1'])/params['s1'] + 1
        
        self.W3 = ((self.W2 / 2) - params['f2'] + 2*params['p2'])/params['s2'] + 1

        self.out_dim = params['d3'] * ((self.W3/2) ** 2)
        print("For debugging out dim before nn Linear W2={}, self.W3={} out {}"
              .format(self.W2,self.W3, self.out_dim))

        # Fully connected layer
        self.fc1 = nn.Linear(int(self.out_dim), 10)

    def forward(self, x):
        #Maybe we use sequential here?
        # Convolution and activation 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pooling 1
        out = self.maxpool1(out)

        # Convolution and activation 1
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pooling 2
        out = self.maxpool2(out)

        # Reshaping to pass it to fc1
        out = out.view(out.size(0), -1)
        # Fully connected layer
        out = self.fc1(out)

        return out

In [30]:
''' Loading the dataset '''
trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

''' Defining batch size and number of iterations and number of epochs based on those'''
batch_size = 100
num_epochs = 10

# setting the seed
torch.manual_seed(1)

# setting input and output dimensions
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
input_dim = 3 * 32 * 32
output_dim = len(CLASSES)

Files already downloaded and verified
Files already downloaded and verified


In [37]:
trainloader, testloader = get_train_test_loader(trainset, testset, batch_size)

parameters = {
    'in_dim': 32, 
    'd1' : 3,
    'd2' : 16,
    'f1' : 5,
    's1' : 1,
    'p1' : 2, # lets try it once
    'd3' : 32,
    'f2' : 5,
    's2' : 1,
    'p2' : 2
}

In [43]:
model = CNNModel(**parameters)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

For debugging out dim before nn Linear W2=32.0, self.W3=16.0 out 2048.0


In [ ]:
'''
    Parameters:
        - criterion, loss function
        - input dimension
        - a numpy array which will save the loss for every epoch
        - the model
        - number of epochs
        - an optimizer
        - and the train loader to iterate through the patterns     
'''
def training(criterion, input_dim, model, num_epochs, optimizer, train_loader, plot=False):
    loss_per_iter = []

    for epoch in range(num_epochs):
        tic = time()
        loss_acc_per_epoch = 0
        for i, (images, labels) in enumerate(train_loader):
            
            # send to device
            images, labels = images.to(device), labels.to(device)

            # reshape images
            images = images.view(-1, model.input_dim)

            # Clearing the gradients.
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)

            # Here the loss function is applied.
            loss = criterion(outputs, labels)

            # We accumulate the loss till the end of the epoch
            loss_acc_per_epoch += loss
            loss_per_iter.append(loss)

            loss.backward()

            optimizer.step()

        print("Finished epoch {:d}/{:d}  in {:f} sec".format(epoch+1, num_epochs, time() - tic))